# Solution for Order Brushing Challenge under the Shopee Code League
#### Vee Lee Koh, Chun Hong Khu, Chun Wye Ng & Li Hong Liew
---
This notebook is a sample solution for the Order Brushing Challenge under the Shopee Code League. <br> The competition was hosted on Kaggle. 


In [0]:
# Import Statements
import pandas as pd
import numpy as np

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Load the comma delimited data file
path = "Order_data.csv"
initial_data = pd.read_csv(path) 
initial_data

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [0]:
# initial_data.describe()

In [0]:
initial_data.dtypes

orderid        int64
shopid         int64
userid         int64
event_time    object
dtype: object

In [0]:
initial_data['event_time'] = pd.to_datetime(initial_data['event_time'])

In [0]:
initial_data.dtypes

orderid                int64
shopid                 int64
userid                 int64
event_time    datetime64[ns]
dtype: object

In [0]:
initial_data.sort_values(by=['shopid', 'event_time'], inplace=True)

In [0]:
initial_data

,orderid,shopid,userid,event_time
59,31086409141107,10009,196962305,2019-12-27 03:06:50
76187,31144571933461,10051,2854032,2019-12-27 19:16:11
9055,31254979546679,10051,48600461,2019-12-29 01:56:19
188513,31195675919209,10061,168750452,2019-12-28 09:27:55
174620,31205132327893,10061,194819216,2019-12-28 12:05:32
...,...,...,...,...
81950,31493203163305,214949521,46269178,2019-12-31 20:06:43
108903,31328775676314,214964814,200983383,2019-12-29 22:26:16
203031,31453591917585,215175775,13688804,2019-12-31 09:06:31
42842,31472077221692,215175775,129266028,2019-12-31 14:14:37


In [0]:
initial_data['shopid'].nunique()

18770

In [0]:
initial_data.shopid.value_counts()

147941492    11703
61556313      6691
54615708      2640
91799978       707
57189823       687
             ...  
163965294        1
17072451         1
194982850        1
160392136        1
189530112        1
Name: shopid, Length: 18770, dtype: int64

There are three shopids with a great deal of orders. Algorithm will take longer during these shopids.

In [0]:
df = initial_data.copy()
df.shape

(222750, 4)

In [0]:
# Retrieve unique shopids sorted by number of transactions ascendingly
unique_shopid = df.shopid.value_counts(ascending=True).reset_index()['index']
# unique_shopid

In [ ]:
### Create solution array
userid = [0] * df['shopid'].nunique()
len(userid)

In [ ]:
### Determine the shopid which has commited order brushing (fraud) 
### and retrieve the userids asscoiated with the fraudulent transactions

idx = 0

# For each unique shopid
for shopid in unique_shopid:
  
  # Tracking purposes
  print (idx, end=' ')

  # Create dataframe for each unique shopid
  shopid_df = df.loc[df.shopid == shopid] 
  # Get total sales per shop
  sales = shopid_df.shape[0]
  # Initiating variables and lists
  highest_sales_prop_shopid = [] # Answer list for each shopid
  orderbrushing_shopid = [] # List for userid assocated with order brushing
  conc_rate = 0 
  start = 0
  counter = 0

  ### Outer time loop
  while start <= sales-2:

    userid_list = []
    # append the first userid
    userid_list.append(shopid_df.iloc[start].userid)
    end = start+1

    # Tracking purposes for the two longest shopids
    if shopid == 147941492 or shopid == 61556313:
      if counter%1000 == 0:
        print (counter, end=' ')
      counter += 1

    ### Inner time loop
    while not end >= sales and (shopid_df.iloc[end].event_time - shopid_df.iloc[start].event_time)/np.timedelta64(1, 'h') <= 1:

      userid_list.append(shopid_df.iloc[end].userid)
      end += 1

    # Calculate conc rate
    unique_buyer = len(set(userid_list))
    conc_rate = len(userid_list) / unique_buyer

    ## Order brushing occured
    if conc_rate >= 3:
      # print(shopid, userid_list)
      start = end # Skip the 1-hour interval
      
      # Append userids to list
      for eachuserid in userid_list:
        orderbrushing_shopid.append(eachuserid)

    else:
      
      # Check if a 1-hour interval exists which satisfly conc rate > 3
      fraud = False
      popped = 0 
      # Stop if less than 3 transactions within an hour
      while len(userid_list) > 3:
        userid_list.pop() # Remove the last userid
        popped += 1
        inter_conc_rate = len(userid_list)/len(set(userid_list)) # Calculate conc rate
        
        if inter_conc_rate >= 3:
          if start !=0 :
            # Check if transactions are within an hour
            if (shopid_df.iloc[end-popped].event_time - shopid_df.iloc[start-1].event_time)/np.timedelta64(1, 'h') > 1:
              fraud = True
          # If start == 0, transactions are within an hour and order brushing occured
          else:
            fraud = True
        
        if fraud:
          # print(shopid, userid_list)
          for eachuserid in userid_list:
            orderbrushing_shopid.append(eachuserid) # Append userids to list
          start = end - popped # Skip transactions that are deemed order brushing
          break
      
      if not fraud:
        start += 1 # Proceed as usual

  # Check if order brushing occured
  if len(orderbrushing_shopid) > 0:

    ## Find mode of element in list as only highest proportion userid will be recorded
    highest_freq = orderbrushing_shopid.count(max(set(orderbrushing_shopid), key = orderbrushing_shopid.count))

    # Get list of unique userids
    unique_userid = list(set(orderbrushing_shopid))
    unique_userid.sort()

    # Calculate frequency of userids and only record the highest proportion one
    for eachuserid in unique_userid:
      freq = orderbrushing_shopid.count(eachuserid)
      if freq == highest_freq:
        highest_sales_prop_shopid.append(eachuserid)

    # Record highest proportion userids in answer array
    highest_sales_shopid_array = np.unique(np.array(highest_sales_prop_shopid))
    userid[idx] = "&".join("{}".format(i) for i in highest_sales_shopid_array)

    # Print answer for viewing
    print('\n', shopid, userid[idx])

  idx += 1 
  


In [0]:
solution = dict(zip(unique_shopid,userid))

In [0]:
solution_df = pd.DataFrame({'shopid':list(solution.keys()),'userid':list(solution.values())})

In [0]:
solution_df.to_csv('solution.csv', header = True, index = False) #saves the csv file